In [5]:
import pandas as pd
import openpyxl
import os
from openpyxl import load_workbook, Workbook
import re

In [4]:
# Ruta del archivo original
archivo = r"C:\Users\9 ----- SIG\Documents\graduados\plantillas\Graduados.xlsx"

# Carpeta donde está el archivo original
carpeta = os.path.dirname(os.path.abspath(archivo))

# Leer datos con pandas
df = pd.read_excel(archivo, sheet_name=0)

# Convertir todas las columnas de fecha a solo fecha (sin hora)
for col in df.select_dtypes(include=["datetime64[ns]"]).columns:
    df[col] = df[col].dt.date

# Iterar por cada año-semestre
for (anio, semestre), datos in df.groupby(["AÑO", "SEMESTRE"]):
    # Cargar una copia del archivo original (mantiene formato y colores)
    wb_copia = load_workbook(archivo)
    hoja_datos = wb_copia.worksheets[0]

    # Borrar las filas viejas de datos (mantener encabezado)
    hoja_datos.delete_rows(2, hoja_datos.max_row)

    # Insertar filas filtradas
    for fila in datos.itertuples(index=False):
        hoja_datos.append(fila)

    # Detectar qué columnas son fechas y aplicar formato dd-mm-yyyy
    for col in datos.select_dtypes(include=["object", "datetime64[ns]"]).columns:
        if "FECHA" in col.upper():  # solo columnas con "FECHA" en el nombre
            idx = datos.columns.get_loc(col) + 1  # posición de la columna (1-based)
            for fila in hoja_datos.iter_rows(min_row=2, max_row=hoja_datos.max_row, min_col=idx, max_col=idx):
                for celda in fila:
                    celda.number_format = "DD/MM/YYYY"

    # Guardar con nombre año-semestre
    nombre_archivo = f"{anio}-{semestre}.xlsx"
    ruta_salida = os.path.join(carpeta, nombre_archivo)
    wb_copia.save(ruta_salida)

print("✅ Archivos generados con formato día-mes-año y respetando la plantilla.")

✅ Archivos generados con formato día-mes-año y respetando la plantilla.


### UNIR EN UN SOLO ARCHIVO 

In [17]:
carpeta = r"C:\Users\9 ----- SIG\Documents\Matriculados\subidos"
archivo_salida = os.path.join(carpeta, "Consolidado_Final_SEGURO.xlsx")

# 2. Listar archivos (patrón 2016-1.xlsx, etc.)
archivos = [f for f in os.listdir(carpeta) if re.match(r"^\d{4}-\d\.xlsx$", f)]
archivos.sort()

if not archivos:
    print("❌ No se encontraron archivos para unir.")
else:
    lista_df = []
    
    print(f"🚀 Iniciando unión de {len(archivos)} archivos (Modo Seguro)...")
    
    for nombre in archivos:
        ruta = os.path.join(carpeta, nombre)
        print(f"Leyendo: {nombre}")
        
        # 'dtype=str' es el truco clave: lee todo como texto. 
        # Así no intenta "adivinar" fechas y no borra nada.
        df_temp = pd.read_excel(ruta, engine="calamine", dtype=str)
        
        # Eliminar solo si la fila está totalmente vacía
        df_temp = df_temp.dropna(how='all')
        
        if not df_temp.empty:
            lista_df.append(df_temp)

    if lista_df:
        # 3. Unir todos los datos
        df_final = pd.concat(lista_df, ignore_index=True)

        # 4. Guardar
        # Usamos engine='xlsxwriter' si lo tienes, o el por defecto de pandas
        df_final.to_excel(archivo_salida, index=False)
        
        print(f"\n✅ ¡Listo! Archivo guardado en: {archivo_salida}")
        print("💡 Nota: Los datos se guardaron como texto para proteger las fechas.")
    else:
        print("⚠ No había datos que unir.")

🚀 Iniciando unión de 20 archivos (Modo Seguro)...
Leyendo: 2016-1.xlsx
Leyendo: 2016-2.xlsx
Leyendo: 2017-1.xlsx
Leyendo: 2017-2.xlsx
Leyendo: 2018-1.xlsx
Leyendo: 2018-2.xlsx
Leyendo: 2019-1.xlsx
Leyendo: 2019-2.xlsx
Leyendo: 2020-1.xlsx
Leyendo: 2020-2.xlsx
Leyendo: 2021-1.xlsx
Leyendo: 2021-2.xlsx
Leyendo: 2022-1.xlsx
Leyendo: 2022-2.xlsx
Leyendo: 2023-1.xlsx
Leyendo: 2023-2.xlsx
Leyendo: 2024-1.xlsx
Leyendo: 2024-2.xlsx
Leyendo: 2025-1.xlsx
Leyendo: 2025-2.xlsx

✅ ¡Listo! Archivo guardado en: C:\Users\9 ----- SIG\Documents\Matriculados\subidos\Consolidado_Final_SEGURO.xlsx
💡 Nota: Los datos se guardaron como texto para proteger las fechas.
